<a href="https://colab.research.google.com/github/mr-nobody15/finetune_llama2/blob/main/evol_dataset_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install libraries
!pip install -q datasets transformers sentence_transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json
from sentence_transformers import SentenceTransformer
import faiss
from datasets import Dataset, DatasetDict
from tqdm.autonotebook import tqdm
import numpy as np



In [ ]:
dataset_path = "/content/converted_alpaca_20k.json"


In [ ]:

# Load the JSON file
with open(dataset_path, 'r') as file:
    data = json.load(file,strict = False)

# # Display the loaded data (for demonstration)
# for item in data:
#     instruction = item.get('instruction')
#     output = item.get('output')
#     print("Instruction:", instruction)
#     print("Output:", output)
#     print()


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import json
from nltk.tokenize import word_tokenize


# Function to count tokens in a text
def count_tokens(text):
    tokens = word_tokenize(text)
    return len(tokens)

# Count total items before filtering
total_items_before = len(data)

# Filter out instructions with less than 100 tokens in the response
filtered_data = [item for item in data if count_tokens(item['output']) >= 100]

# Count total items after filtering
total_items_after = len(filtered_data)

# Display total counts
print("Total items before filtering:", total_items_before)
print("Total items after filtering:", total_items_after)


Total items before filtering: 20022
Total items after filtering: 1992


In [ ]:

# Display the filtered data (for demonstration)
for item in filtered_data[:2]:
    instruction = item.get('instruction')
    output = item.get('output')
    print("Instruction:", instruction)
    print("Output:", output)
    print()


Instruction: Develop a classification algorithm in Python to predict whether a bird is a hawk or a falcon.
Output: import pandas as pd
import numpy as np

# Define features
features = ["wing_shape", "size", "tails_length", "color_pattern"] 

# Load the data
data = pd.read_csv("birds.csv")

# Preprocess the data to generate feature vectors
X = np.array(data[features]) 

# Make a target vector 
y = np.array(data["species"]) 

# Divide the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a classification model
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Generate evaluation metrics
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

print("Model accuracy: %.2f" % accuracy)

Instruction: Create a C++ progra

In [ ]:
type(filtered_data)

list

In [ ]:
# Assuming filtered_data is a list of dictionaries
instructions = [item['instruction'] for item in filtered_data]
outputs = [item['output'] for item in filtered_data]

# Getting the lengths
len(instructions), len(outputs)


(1992, 1992)

In [ ]:
filtered_data_df = pd.DataFrame(filtered_data)
filtered_data_df.head()

,instruction,output
0,Develop a classification algorithm in Python t...,import pandas as pd\nimport numpy as np\n\n# D...
1,Create a C++ program for randomly shuffling an...,#include <iostream>\n#include <algorithm>\n#in...
2,Create a C program that prints out the Fibonac...,#include <stdio.h>\n\nint main()\n{\n int i...
3,Create a C++ program that sorts a given array ...,#include<iostream>\nusing namespace std;\n \ni...
4,Write a HTML code that create a form for user ...,"<form action="""">\n <label for=""street"">Stre..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# Concatenate instructions and outputs
all_texts = instructions + outputs

# Vectorize text data (you might need to preprocess texts accordingly)
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(all_texts)

# Calculate cosine similarity
cos_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Set threshold for similarity
threshold = 0.961

# Find duplicate indices
duplicates = np.where(cos_sim_matrix > threshold)

# Remove duplicates
deduplicated_indices = set()
for i, j in zip(*duplicates):
    # Keep only one instance from each pair of highly similar instances
    if i != j:
        deduplicated_indices.add(min(i, j))

deduplicated_data = [all_texts[i] for i in deduplicated_indices]


In [ ]:
def deduplicate_dataset(dataset, model: str, threshold: float):
    sentence_model = SentenceTransformer(model)

    instructions = dataset['instruction'].tolist()
    outputs = dataset['output'].tolist()

    print("Converting text to embeddings...")
    embeddings = sentence_model.encode(outputs, show_progress_bar=True)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    index.add(normalized_embeddings)

    print("Filtering out near-duplicates...")
    D, I = index.search(normalized_embeddings, k=2)
    to_keep = []

    for i in tqdm(range(len(embeddings)), desc="Filtering"):
        # If the second closest vector (D[i, 1]) has cosine similarity above the threshold
        if D[i, 1] >= threshold:
            # Check if either the current item or its nearest neighbor is already in the to_keep list
            nearest_neighbor = I[i, 1]
            if i not in to_keep and nearest_neighbor not in to_keep:
                # If not, add the current item to the list
                to_keep.append(i)
        else:
            # If the similarity is below the threshold, always keep the current item
            to_keep.append(i)

    deduplicated_indices = dataset.iloc[to_keep]
    return deduplicated_indices

deduped_dataset = deduplicate_dataset(filtered_data_df,"thenlper/gte-base", 0.961)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/219M [00:00<?, ?B/s]

...

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

Converting text to embeddings...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Filtering out near-duplicates...


Filtering:   0%|          | 0/1992 [00:00<?, ?it/s]

In [ ]:
type(deduped_dataset)

pandas.core.frame.DataFrame

In [ ]:
from datasets import Dataset

# Convert DataFrame to Hugging Face Dataset
hf_deduped_dataset = Dataset.from_pandas(deduped_dataset)


In [ ]:
!pip install huggingface-cli


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame
df = deduped_dataset
# Convert DataFrame to JSONL (Newline delimited JSON)
with open('train.jsonl', 'w') as file:
    for index, row in df.iterrows():
        row_dict = row.to_dict()
        json_string = pd.io.json.dumps(row_dict)  # Convert row to JSON string
        file.write(json_string + '\n')  # Write JSON string followed by newline


In [ ]:
from datasets import load_dataset
train1 = load_dataset('Akil15/evol_20k_filter',split = 'train')
train1

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['instruction', 'output'],
    num_rows: 1855
})

In [ ]:
train1[0]

{'instruction': 'Develop a classification algorithm in Python to predict whether a bird is a hawk or a falcon.',
 'output': 'import pandas as pd\nimport numpy as np\n\n# Define features\nfeatures = ["wing_shape", "size", "tails_length", "color_pattern"] \n\n# Load the data\ndata = pd.read_csv("birds.csv")\n\n# Preprocess the data to generate feature vectors\nX = np.array(data[features]) \n\n# Make a target vector \ny = np.array(data["species"]) \n\n# Divide the data into training and test sets\nfrom sklearn.model_selection import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Train a classification model\nfrom sklearn.tree import DecisionTreeClassifier\nmodel = DecisionTreeClassifier()\nmodel.fit(X_train, y_train)\n\n# Make predictions\ny_pred = model.predict(X_test)\n\n# Generate evaluation metrics\nfrom sklearn.metrics import accuracy_score\naccuracy = accuracy_score(y_test, y_pred)\n\nprint("Model accuracy: %.2f" % ac